# 生成 AI エージェント

https://docs.cohere.com/v2/docs/building-an-agent-with-cohere

In [ ]:
import logging
import os
#import yaml
import json
import numpy as np
from typing import List
from dotenv import load_dotenv
import cohere

load_dotenv()
co = cohere.ClientV2(api_key=os.environ['COHERE_API_KEY'])

# ツールの作成